In [1]:
import io
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
%matplotlib inline
from nltk import FreqDist
from PIL import Image
import matplotlib.pyplot as plt
from wordcloud import WordCloud 
import nltk
from nltk import FreqDist

In [3]:
with open("textFile.txt", "r", encoding='utf8') as f:
    file = f.read()

In [8]:
raw = file.split("CHAPTER")[1:]
print("Number of chapters in the book: ",len(raw))

Number of chapters in the book:  50


In [9]:
book_df = pd.DataFrame({"chapter": raw})

In [10]:
chapters = book_df.copy()
chapters['chapter_num'] = chapters.index+1

In [11]:
chapters

,chapter,chapter_num
0,1\n\n\nThe family of Dashwood had long been s...,1
1,2\n\n\nMrs. John Dashwood now installed herse...,2
2,3\n\n\nMrs. Dashwood remained at Norland seve...,3
3,"4\n\n\n""What a pity it is, Elinor,"" said Mari...",4
4,"5\n\n\nNo sooner was her answer dispatched, t...",5
5,6\n\n\nThe first part of their journey was pe...,6
6,7\n\n\nBarton Park was about half a mile from...,7
7,8\n\n\nMrs. Jennings was a widow with an ampl...,8
8,9\n\n\nThe Dashwoods were now settled at Bart...,9
9,"10\n\n\nMarianne's preserver, as Margaret, wi...",10


In [12]:
def unnest_tokens(dataframe, column, tokenizer, new_column='word'): 
    """
    Applies a tokenizer to a column and then unnest the dataframe
    
    dataframe: dataframe with text
    column: name of the column with the text
    new_column: what you want the column of words to be called
    tokenizer: a function to be used to tokenize the text
    """ 
    new_column_num = new_column + "_num"
    df = ( dataframe[column]
              .apply(tokenizer)
              .apply(pd.Series)
              .stack()
              .reset_index(level=1)
              .rename(columns={0: new_column, "level_1":new_column_num})
              .join(dataframe.drop(column, axis=1), how='left')
              .reset_index(drop=True)
         )
    df[new_column_num] = pd.to_numeric(df[new_column_num])
    df[new_column_num] = df[new_column_num] + 1
    return df

In [14]:
chapters['chapter'] = chapters.chapter.str.replace("\n", " ")

## Get Sentences

In [15]:
import spacy
nlp = spacy.load('en')

In [16]:
def sentence_tokenizer(text):
    doc = nlp(text)
    sentences = [sent.string.strip() for sent in doc.sents]
    return sentences

In [17]:
sentences = unnest_tokens(chapters, "chapter", sentence_tokenizer, "sentence")

In [18]:
sentences

,sentence_num,sentence,chapter_num
0,1,1,1
1,2,The family of Dashwood had long been settled i...,1
2,3,"Their estate was large, and their residence wa...",1
3,4,The late owner of this estate was a single man...,1
4,5,"But her death, which happened ten years before...",1
...,...,...,...
6104,61,"His wife was not always out of humour, nor his...",50
6105,62,"For Marianne, however--in spite of his incivil...",50
6106,63,Mrs. Dashwood was prudent enough to remain at ...,50
6107,64,"Between Barton and Delaford, there was that co...",50


## Get Words

In [19]:
def word_tokenizer(text):
    doc = nlp(text)
    words = [[token.text, token.lemma_, token.pos_, token.is_stop] for token in doc]
    return words

In [20]:
words = unnest_tokens(sentences, "sentence", word_tokenizer, "word")

In [21]:
words.head()

,word_num,word,sentence_num,chapter_num
0,1,"[1, 1, X, False]",1,1
1,1,"[The, the, DET, True]",2,1
2,2,"[family, family, NOUN, False]",2,1
3,3,"[of, of, ADP, True]",2,1
4,4,"[Dashwood, Dashwood, PROPN, False]",2,1


In [22]:
words[['word', 'lemma', 'POS', "stop"]] = pd.DataFrame(words.word.values.tolist())

In [23]:
words.head()

,word_num,word,sentence_num,chapter_num,lemma,POS,stop
0,1,1,1,1,1,X,False
1,1,The,2,1,the,DET,True
2,2,family,2,1,family,NOUN,False
3,3,of,2,1,of,ADP,True
4,4,Dashwood,2,1,Dashwood,PROPN,False


In [24]:
words.to_csv("words.csv")